# MLPR assignment 2

**Background**: In this assignment you will perform some preliminary analysis of a dataset from the UCI machine learning repository. Some features have been extracted from slices of CT medical scans. There is a regression task: attempt to guess the location of a slice in the body from features of the scan. A little more information about the data is available online[1](https://mlpr.inf.ed.ac.uk/2023/notes/as2_sliceloc.html#fn2):
https://archive.ics.uci.edu/ml/datasets/Relative+location+of+CT+slices+on+axial+axis
However, you should use the processed version of the dataset given below.

[Download the data here](https://media.inf.ed.ac.uk/teaching/courses/mlpr/data/ct_data.npz) (26 MB).

The patient IDs were removed from this version of the data, leaving 384 input features which were put in each of the “`X_...`” arrays. The corresponding CT scan slice location has been put in the “`y_...`” arrays. We shifted and scaled the “`y_...`” location values for the version of the data that you are using. The shift and scaling was chosen to make the training locations have zero mean and unit variance.

The first 73 patients were put in the `_train` arrays, the next 12 in the `_val` arrays, and the final 12 in the `_test` arrays. Please use this training, validation, test split as given. Do not shuffle the data further in this assignment.

**The code**: We provide [support code](https://mlpr.inf.ed.ac.uk/2023/notes/ct_support_code.py) along with this assignment, which you will need for the questions below. Do not copy this code into the answer boxes; your code should `import` it.

1. **Getting started [15 marks]:**
   ***As with each question, please report the few lines of code required to perform each requested step. When a question says to report a number, your code should print it, and manually include the resulting number as part of your answer.***
   
   Load the data into Python (your code can assume this step has been done):

In [1]:
import numpy as np
data = np.load('ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

   a. Verify that (up to numerical rounding errors) the mean of the training positions in `y_train` is zero. The mean of the 5,785 positions in the `y_val` array is not zero. Report its mean with a “standard error”, temporarily assuming that each entry is independent. For comparison, also report the mean with a standard error of the first 5,785 entries in the `y_train`. Explain how your results demonstrate that these standard error bars do not reliably indicate what the average of locations in future CT slice data will be. Why are standard error bars misleading here?

   **Answer:**

In [2]:
# Verify the mean of training positions
print(np.allclose(np.mean(y_train), 0))
# Verify the mean of validation positions
print(np.allclose(np.mean(y_val), 0))
# Mean with standard error of validation positions
print(np.mean(y_val), np.std(y_val) / np.sqrt(len(y_val)))
# Mean with standard error of first 5785 entries in training positions
y_part = y_train[: 5785]
print(np.mean(y_part), np.std(y_part) / np.sqrt(len(y_part)))

True
False
-0.2160085093241599 0.012903383410668334
-0.44247687859693674 0.01192627246273395


The results show that:

Mean for `y_val`: -0.2160085093241599
Standard Error for `y_val`: 0.012903383410668334
Mean for first 5785 entries of `y_train`: -0.44247687859693674
Standard Error for first 5785 entries of `y_train`: 0.01192627246273395

Therefore, the error bar for `y_val` is $\mu_1=\bar{y}_\text{val}\pm\hat{\sigma}_\text{val}/\sqrt{N_\text{val}}$, $-0.2031<\mu_1<-0.2289$, and the error bar for forst 5785 entries of y_train is $\mu_2=\bar{y}_\text{tr}'\pm\hat{\sigma}_\text{tr}'/\sqrt{N_\text{tr}'}$, $-0.4306<\mu_2<-0.4544$. Both of them do not include 0, which is the true mean of the positions, according to the shifting and scaling operations done before verifying the data, thus these standard error bars can not indicate the true average of locations in the CT slice data, as well as the future data.

The standard error bars are misleading here because the sampling of neither validation set nor the first 5785 entries of the training set conforms to the real position data distribution. Therefore the estimators of mu are biased, even with error bars.

   b. Some of the input features are constants: they take on the same value for every training example. Identify these features, and remove them from the input matrices in the training, validation, and testing sets.
   
   Moreover, some of the input features are duplicates: some of the columns in the training set are identical. For each training set column, discard any later columns that are identical. Discard the same columns from the validation and testing sets.
   
   **Use these modified input arrays for the rest of the assignment.** Keep the names of the arrays the same (`X_train`, etc.), so we know what they’re called. You should not duplicate the code from this part in future questions. We will assume it has been run, and that the modified data are available.
   
   **Warning**: As in the real world, mistakes at this stage would invalidate all of your results. We strongly recommend checking your code, for example on small test examples where you can see what it’s doing.
   
   Report which columns of the `X_...` arrays you remove at each of the two stages. Report these as 0-based indexes. (For the second stage, you might report indexes in the original array, or after you did the first stage. It doesn’t matter, as long as your code is clear and correct.)

   **Answer:**

In [3]:
# Identify the input features with constant values and discard them
ft_const = np.all(X_train == X_train[0, ], axis=0)
# Update the input data matricies
X_train = X_train[: , ~ ft_const]; X_val = X_val[: , ~ ft_const]; X_test = X_test[: , ~ ft_const]

# Identify the dupicated input features and keep only the first
X_train_, ft_unique = np.unique(X_train, axis=1, return_index=True)
# Calculate the dicarded indices in this satge
ft_dup = np.setdiff1d(np.arange(X_train.shape[1]), ft_unique)
# Update the input data matricies
X_train = X_train[: , ft_unique]; X_val = X_val[: , ft_unique]; X_test = X_test[: , ft_unique]

# Report the dicarded columns (with stage-wise indices)
print(np.where(ft_const))
print(ft_dup)

(array([ 59,  69, 179, 189, 351], dtype=int64),)
[ 76  77 185 195 283 354]


The column indices removed in the first stage is: $59,  69, 179, 189, 351$, and the ones removed in the second stage is: $76, 77, 185, 195, 283, 354$. Note that the latter ones are indices after doing the first stage.

2. **Linear regression baseline [15 marks]:**
   Using `numpy.linalg.lstsq`, write a short function “`fit_linreg(X, yy, alpha)`” that fits the linear regression model 
   
   $$f(\mathbf{x};\mathbf{w},b) = \mathbf{w}^\top\mathbf{x} + b,$$
   
   by minimizing the cost function:
   
   $$E(\mathbf{w}, b) = \alpha\mathbf{w}^\top\mathbf{w} + \sum_n (f(\mathbf{x}^{(n)};\mathbf{w},b) - y^{(n)})^2,$$
   
   with regularization constant $\alpha$. As discussed in the lecture materials, fitting a bias parameter $b$ and incorporating the regularization constant can both be achieved by augmenting the original data arrays. Use a data augmentation approach that maintains the numerical stability of the underlying `lstsq` solver, rather than a ‘normal equations’ approach. You should only regularize the weights $\mathbf{w}$ and not the bias $b$.
   
   (In the lecture materials we used $\lambda$ for the regularization constant, matching Murphy and others. However, `lambda` is a reserved word in Python, so we swapped to `alpha` for our code.)
   
   Use your function to fit weights and a bias to `X_train` and `y_train`. Use $\alpha=30$.
   
   We can fit the same model with a gradient-based optimizer. The support code has a function `fit_linreg_gradopt`, which you should look at and try.
   
   Report the root-mean-square errors (RMSE) on the training and validation sets for the parameters fitted using both your `fit_linreg` and the provided `fit_linreg_gradopt`. Do you get exactly the same results? Why or why not?

   **Answer:**

In [4]:
def fit_linreg(X, yy, alpha=30):
    '''
    fit a regularized linear regression model with data augmentation and lstsq solver.
     Use data augnentation approach and solve y' ~ X'w with a lstsq solver,
     minimizing the regularized least squares cost (bias not regularized):

       - np.sum(-yy*(np.dot(X, ww) + bb)) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    '''
    D = X.shape[0]
    # Construct the design matrix
    phi = np.hstack([np.ones((D, 1)), X])
    # Construct the augmenting diagnal matrix
    diag_aug = np.sqrt(alpha) * np.identity(phi.shape[1])
    # not regularizing bias, make the position corresponding to bias 0
    diag_aug[0, 0] = 0
    # Concatenate the augmenting matrix to the original design matrix vertically
    phi_reg = np.vstack([phi, diag_aug])
    # Concatenate a 0 vector to the target vector
    y_reg = np.hstack([yy, np.zeros(phi.shape[1])])

    # generate the fitted weight with lstsq
    w_fit = np.linalg.lstsq(phi_reg, y_reg, rcond=None)[0]

    # separately returning weights and bias
    ww, bb = w_fit[1: ], w_fit[0]

    return ww, bb

from support_code import *

def get_RMSE_lin(ww, bb, X, yy):
    '''
    get the root-mean-square-loss (RMSE) of given data for a linear fit 

     RMSE = np.sqrt(np.mean((yy - yy_fit) ** 2))
     Inputs:
             ww D, fitted weights
             bb    fitted bias
             X N,D design matrix of input features
            yy N,  real-valued targets

     Outputs:
            RMSE   RMSE of given data for the linear fit 
    '''
    # calculate the predicted targets under the fitted values
    yy_fit = X.dot(ww) + bb
    # return the RMSE of given data for the linear fit
    return np.sqrt(np.mean((yy - yy_fit) ** 2))

# Specific the regularization constant alpha as 30
alpha = 30
# Fit the linear model using self-implemented fit_linreg
ww_1, bb_1 = fit_linreg(X_train, y_train, alpha)
# Fit the linear model using provided fit_linreg_gradopt
ww_2, bb_2 = fit_linreg_gradopt(X_train, y_train, alpha)
# Report the RMSE on the training set for parameters fitted with fit_linreg
print(get_RMSE_lin(ww_1, bb_1, X_train, y_train))
# Report the RMSE on the training set for parameters fitted with fit_linreg_gradopt
print(get_RMSE_lin(ww_2, bb_2, X_train, y_train))
# Report the RMSE on the validation set for parameters fitted with fit_linreg
print(get_RMSE_lin(ww_1, bb_1, X_val, y_val))
# Report the RMSE on the validation set for parameters fitted with fit_linreg_gradopt
print(get_RMSE_lin(ww_2, bb_2, X_val, y_val))

0.3567565397204055
0.3567563054700464
0.4230521968394695
0.4230582267771714


The RMSEs on the training set and validation set for parameters fitted with self-implemented `fit_linreg` is: $0.356757$ and $0.423052$;

The RMSEs on the training set and validation set for parameters fitted with provided fit_linreg_gradopt is: $0.356758$ and $0.423050$.

From which we can see that the two functions get approximately the same results. This is because in `fit_linreg`, `lstsq` directly uses least squares to calculate the fitted values, whose mathematical forms could be derived from calculating the derivative of the least squares loss in the linear case. 

`fit_linreg_gradopt` instead explicitly uses the gradient-based method (minimize with L-BFGS-B) to find the fitted values, which is equivalent to least squares in the linear case. The latter method could be used in a more general case.

The minor differences between the 2 sets of results could be due to that `lstsq` has more numerical stability, which in comparison iterative gradient-based methods lack.

3. **Inverted classification tasks [20 marks]:**
   
   It is often easier to work with binary data than real-valued data: we don’t have to think so hard about how the values might be distributed, and how we might process them. We will invent some binary classification tasks, and fit these.
   
   We will pick 20 positions within the range of training positions, and use each of these to define a classification task:
   
   ```Python
   K = 20 # number of thresholded classification problems to fit
   mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
   thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
   for kk in range(K):
		labels = y_train > thresholds[kk]
		# ... fit logistic regression to these labels
   ```

   The logistic regression cost function and gradients are provided with the assignment in the function `logreg_cost`. It is analogous to the `linreg_cost` function for least-squares regression, which is used by the `fit_linreg_gradopt` function that you used earlier.
   
   Fit logistic regression to each of the 20 classification tasks above with $\alpha=30$.
   
   Given a feature vector, we can now obtain 20 different probabilities, the predictions of the 20 logistic regression models. Transform both the training and validation input matrices into new matrices with 20 columns, containing the probabilities from the 20 logistic regression models. You don’t need to loop over the rows of `X_train` or `X_val`, you can use array-based operations to make the logistic regression predictions for every datapoint at once.
   
   Fit a regularized linear regression model ($\alpha=30$) to your transformed 20-dimensional training set. Report the training and validation root mean square errors (RMSE) of this model.

   **Answer:**

In [ ]:
def fit_logreg_gradopt(X, yy, alpha):
    """
    fit a regularized logistic regression model with gradient opt

         ww, bb = fit_logreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized negative log likelihood
       least squares cost:

       - np.sum(np.log(1 / (1 + np.exp(-yy*(np.dot(X, ww) + bb))))) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    D = X.shape[1]
    # create the arguments needed for calculating costs in minimize
    args = (X, yy, alpha)
    # generate the initializing values for ww and bb
    init = (np.zeros(D), np.array(0))
    # fit the model with minimize_list, using cost function logreg_cost
    # functions imported in Q2
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb

# number of thresholded classification problems to fit
K = 20
# Caculate the range of y_train values and the step size w.r.t K thresholds
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
# Create the threshold ndarray w.r.t. number K 
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)

## logistic regression fit
# Initialize the ndarray to store the logistic regression result w.r.t each class
prob_train = np.zeros((len(y_train), K))
prob_val = np.zeros((len(y_val), K))

# Initialize the logistic regression parameters for K classes
ww_V = np.zeros((K, X_train.shape[1]))
bb_k = np.zeros(K)

# Loop over K logistical regression problems
for kk in range(K):
    # Generate the new labels under logistic regression settings
    labels = y_train > thresholds[kk]
    # fit logistic regression to these labels (using alpha from Q2)
    ww_V[kk, :], bb_k[kk] = fit_logreg_gradopt(X_train, labels, alpha)
    # Generate the predicted probabilities for this class with fitted parameters
    prob_train[:, kk] = 1 / (1 + np.exp(- (np.dot(X_train, ww_V[kk, :]) + bb_k[kk])))
    prob_val[:, kk] = 1 / (1 + np.exp(- (np.dot(X_val, ww_V[kk, :]) + bb_k[kk])))

# linear regression fit to the probabilities for each logistic regression and the final label
# (fit_linreg_gradopt imported in Q2)
ww_lin, bb_lin = fit_linreg_gradopt(prob_train, y_train, alpha)
# calculate RMSE of the training set and validation set with get_RMSE_lin (from Q2)
print(get_RMSE_lin(ww_lin, bb_lin, prob_train, y_train))
print(get_RMSE_lin(ww_lin, bb_lin, prob_val, y_val))

1. RMSE for training set: $0.1544$;

2. RMSE for validation set: $0.2542$.

4. **Small neural network [10 marks]:**
   
   In Question 3 you fitted a small neural network. The logistic regression classifiers are sigmoidal hidden units, and a linear output unit predicts the outputs. However, you didn’t fit the parameters jointly to the obvious least squares cost function. A least squares cost function and gradients for this neural network are implemented in the `nn_cost` function provided.
   
   Try fitting the neural network model to the training set, with a) a sensible random initialization of the parameters; b) the parameters initialized using the fits made in Q3.
   
   Does one initialization strategy work better than the other? Does fitting the neural network jointly work better than the procedure in Q3? Your explanation should include any numbers that your answer is based on.

**Answer:**

In [ ]:
# Set seed to get consistent results for comparison
np.random.seed(10)
def fit_nn(X, yy, alpha, init):
    """
    fit a regularized neural network model with gradient opt
     Find parameters by using a gradient-based optimizer
     (minimize_list) to minimize the regularized least squares cost:

       np.dot(np.dot(1 / (1 + np.exp(-np.dot(X, V.T) + bk)), ww) + bb - yy) 
        + alpha*(np.sum(V*V) + np.dot(ww,ww))

     Inputs:
             X N,D input design matrix
            yy N,  regression targets
         alpha     scalar regularization for weights

     Outputs:
            ww K,  hidden-output weights
            bb     scalar output bias
            V K,D hidden-input weights
            bk K,  hidden biases
    """
    # Create the arguments needed for calculating costs in minimize
    args = (X, yy, alpha)
    # fit the model with minimize_list, using cost function nn_cost
    # functions imported in Q2
    ww, bb, V, bk = minimize_list(nn_cost, init, args)
    return ww, bb, V, bk

def get_RMSE_nn(params, X, yy):
    '''
    get the root-mean-square-loss (RMSE) of given data for a neural network

     RMSE = np.sqrt(np.mean((yy - yy_fit) ** 2))
     Inputs:
             ww D,  fitted weights
             bb     fitted bias
              V K,D hidden-input weights
             bk K,  hidden biases
              X N,D design matrix of input features
             yy N,  real-valued targets

     Outputs:
            RMSE   RMSE of given data for the nn
    '''
    # calculate the predicted targets under the fitted values
    # (nn_cost imported in Q2)
    nn_fit = nn_cost(params, X)
    # return the RMSE of given data for the neural network
    return np.sqrt(np.mean((yy - nn_fit) ** 2))

D = X_train.shape[1]
# Create a sensible random initialization of the parameters (uniform(-1, 1) here)
uni_init = (np.random.uniform(-1, 1, K), np.random.uniform(-1, 1),
            np.random.uniform(-1, 1, (K, D)), np.random.uniform(-1, 1, K))
# Retrieve the parameters pretrained in logistic regressions from Q3 for initialization
pretrain_init = (ww_lin, bb_lin, ww_V, bb_k)

# nn fit with sensible random initialization
params_nn_a = fit_nn(X_train, y_train, alpha, init=uni_init)
# Report the RMSE of training set
print(get_RMSE_nn(params_nn_a, X_train, y_train))
# Report the RMSE of validation set
RMSE_nn_val = get_RMSE_nn(params_nn_a, X_val, y_val)
print(RMSE_nn_val)

# nn fit with pretrained parameter initialization
params_nn_b = fit_nn(X_train, y_train, alpha, init=pretrain_init)
# Report the RMSE of training set
print(get_RMSE_nn(params_nn_b, X_train, y_train))
# Report the RMSE of validation set
print(get_RMSE_nn(params_nn_b, X_val, y_val))

1. RMSE for training set with uniform initialized nn: $0.1434$;

2. RMSE for validation set with uniform initialized nn: $0.2751$;

3. RMSE for training set with Q3-pretrained initialized nn: $0.1395$;

4. RMSE for validation set with Q3-pretrained initialized nn: $0.2700$.

Yes, as the RMSEs of the pretrained parameters initialized method is smaller than those of the uniformly initialized method on both training and validation set, overally strategy b works better than strategy a. This is because the pretrained parameters contains more pre-dominant knowledge than the uniformly initialized parameters about the fitting process, thus is easier for gradient optimizers to find their ways to the optima.

For the second question, though the RMSEs of the pretrained parameters initialized method is smaller than those of the uniformly initialized method on the training set, it's inversed on the validation set, indicating that the procedure in Q3 has stronger generalizing ability. Thererfore, no, the nn method jointly doesn't work better than the procedure in Q3. This is because that the stage-wise fitting is easier to optimize with less parameters in each stage, and the nn fitting needs more effort to reach the optima. As we can check from the minimize result message, the optimization in nn stops due to "total no. of iterations reached limit", thus it might be underfitted to some extent.

5. **Bayesian optimisation [20 marks]:**
   
   A popular application area of Gaussian processes is Bayesian optimisation, where the uncertainty in the probabilistic model is used to guide the optimisation of a function. Here we will use Bayesian optimisation with Gaussian processes for choosing the regularisation parameter $\alpha$. (We would normally use Bayesian optimisation when optimizing more than one parameter.)
   
   Gaussian processes are used to represent our belief about an unknown function. In this case, the function we are interested in is the neural network’s validation log root mean square error (log RMSE) as a function of the regularisation paramter $\alpha$. In Bayesian optimisation, it is common to maximise the unknown function, so we will maximise the negative log RMSE.
   
   We start with a Gaussian process prior over this function. As we observe the actual log RMSEs for particular $\alpha$’s we update our belief about the function by calculating the Gaussian process posterior.
   
   Besides the Gaussian process framework that you’re already familiar with, Bayesian optimisation involves a so-called acquisition function. Given our Gaussian process posterior model, we use this function to decide which parameter to query next. The acquisition function describes how useful we think it will be to try a given $\alpha$, while considering the uncertainty that is represented in our posterior belief.
   
   There are many popular acquisition functions in Bayesian optimisation. One example is the probability of improvement. Suppose we have observed $y^{(1)}$ to $y^{(1)}$ (here negative log RMSE at locations $\alpha^{(1)}$ to $\alpha^{(N)}$). Then the function takes the following form:

   $$\notag\mathit{PI}(\alpha) = \Phi\left(\frac{\mu(\alpha) - \text{max}(y^{(1)},\dots,y^{N})}{\sigma(\alpha)}\right),$$

   where $\mu(\alpha)$ is the Gaussian process posterior mean at location $\alpha$, $\sigma(\alpha)$ is the posterior standard deviation at location $\alpha$, and $\Phi$ denotes the cumulative density function of the Gaussian with mean 0 and variance 1.
   
   We pick the next regularization constant $\alpha^{(N+1)}$ by maximizing the acquisition function:

   $$\notag\alpha^{(N+1)} = \argmax_\alpha\mathit{PI}(\alpha).$$

   We then evaluate our model for this regularization parameter and update our posterior about the unknown function that maps $\alpha$ to negative log RMSE. We repeat the procedure multiple times and then pick the parameter that yielded the best performance $y$.
   
   Write a function `train_nn_reg` that trains the neural network from Q4 for a given $\alpha$ parameter and returns the validation RMSE.
   
   Consider $\alpha$ on the range `np.arange(0, 50, 0.02)`. Pick three values from this set for $\alpha$ as training locations and use `train_nn_reg` on these locations. Use the remaining locations as values that you will consider with the acquisition function.
   
   Take the performance of the neural network that you trained in Q4 a) as a baseline. Subtract your $\alpha$-observed log RMSEs from the log of this baseline and take the resulting values as $y^{(1)}$ to $y^{(3)}$. Then calculate the Gaussian process posterior for these observations. To do so, use `gp_post_par` from the support code. Use the default parameters for `sigma_y`, `ell` and `sigma_f`.
   
   Implement the probability of improvement acquisition function. You can use `scipy.stats.norm.cdf` to calculate $\Phi$. With this acquisition function, iteratively pick new $\alpha$’s, re-train and evaluate each new model with `train_nn_reg` and update your posterior with `gp_post_par`. Do five of these iterations.
   
   Report the maximum probability of improvement together with its $\alpha$ for each of the five iterations. Also report the best $\alpha$ that this procedure found, and its validation and test RMSEs. Have we improved the model?
   
   In this question, the function we are optimizing is the neural network’s validation log RMSE as a function of the regularisation parameter $\alpha$. Where is the observation noise coming from?

   **Answer:**

In [ ]:
from scipy.stats import norm
from tqdm import tqdm

def train_nn_reg(X, yy, alpha, init):
    '''
    train a nn for given data with fit_nn and get the RMSE for X and yy

     Inputs:
              X N,D design matrix of input features
             yy N,  real-valued targets
            alpha   scalar regularization for weights
             init   a four-element tuple containing the initialization for parameters
     Outputs:
            RMSE   RMSE of given data for fitted nn
    '''
    # fit the neural network (fit_nn from Q4)
    params = fit_nn(X_train, y_train, alpha, init)
    # calculate the RMSE on X and yy (get_RMSE_nn from Q4)
    return get_RMSE_nn(params, X, yy)

def getPI(mu_a, cov_a, yy):
    '''
    Obtain the probability of improvement (PI) value for given unobserved alphas
     and the observed y values

     PI = Phi((mu(a)-max(yy))/sigma(a)), 
     
     Phi is the cumulative density function of standard Gaussian.
     Inputs:
           mu_a  M,   mean values for the unobserved alphas
           cov_a M,M  covariance matrix for the unobserved alphas
            yy   M,   y values for the observed alphas
     Outputs:
            PI        the PI value
    '''
    return norm.cdf((mu_a - np.max(yy)) / np.sqrt(np.diag(cov_a)))

def bayes_optim(a_rest, a_obs, RMSE_base, init=pretrain_init, num_iter=5):
    '''
    Apply a bayes optimization on a neural network to search for the best
     regularization constant, alpha

     Inputs:
          a_rest C   design matrix of input features
           a_obs T-C real-valued targets
       RMSE_base   scalar baseline RMSE
            init   a four-element tuple containing the initialization for nn parameters
                    default: pretrain_init from Q4
        num_iter   the iteration go through before returning the best alpha
                    default: 5
     Outputs:
      best_alpha   the best alpha generated by bayes optimization
             PIs   the maximum probability of improvement (PI) for each iteration
      new_alphas   corresponding alphas for each iteration 
    '''
    # Obtain the y values for the beginning observed alphas
    y_obs = []
    for i in tqdm(range(len(a_obs))):
        y_obs.append(np.log(RMSE_base) - np.log(train_nn_reg(X_val, y_val, a_obs[i], init)))
    
    # Create PIs to record the max PI for each iteration
    PIs = []
    for i in tqdm(range(num_iter)):
        # Calculate the gaussian process prosterior for the rest alphas
        # (gp_post_par imported from support_code)
        rest_mu, rest_cov = gp_post_par(a_rest, a_obs, np.array(y_obs))

        # Obtain the PI values for the unobserved alphas
        PI = getPI(rest_mu, rest_cov, y_obs)

        # pick the next alpha accordding to the PI values for each of the unobserved alphas
        # and update a_obs, a_rest, and PIs
        a_next_id = np.argmax(PI)
        a_obs = np.append(a_obs, a_rest[a_next_id])
        a_rest = np.delete(a_rest, a_next_id)
        PIs.append(PI[a_next_id])

        # Observe the y value for the picked alpha
        y_obs.append(np.log(RMSE_base) - np.log(train_nn_reg(X_val, y_val, a_obs[-1], init)))
    
    # Retrieve the best alpha according by finding which gives the maximum y_obs
    assert len(a_obs) == len(y_obs)
    best_a = a_obs[np.argmax(y_obs)]

    # a_obs[3:] corresponds to the newly added alphas
    return best_a, PIs, a_obs[3:]

# Create the possible alphas on the specific range
alphas = np.arange(0, 50, 0.02)
# randomly select three beginning alphas from alphas
alpha_obs = np.random.choice(alphas, 3, replace=False)
# Report the beginning alphas
print("Beginning alphas: ", alpha_obs)
# Delete the observed alphas from the unobserved ones to get alpha_rest
alpha_rest = np.setdiff1d(alphas, alpha_obs)
# Do bayesian optimization on the given setting (RMSE_nn_val, uni_init from Q4)
best_alpha, PIs, new_alphas = bayes_optim(alpha_rest, alpha_obs, RMSE_nn_val, init=pretrain_init)

# Report the maximum probability of improvement together with its alpha
# for each of the five iterations
print("PIs:         ", PIs)
print("New alphas:  ", new_alphas)
# Report the best alpha
print("Best alpha:  ", best_alpha)

# Report the RMSE on validation set and test set based on the optimized alpha
print(train_nn_reg(X_val, y_val, best_alpha, pretrain_init))
print(train_nn_reg(X_test, y_test, best_alpha, pretrain_init))

1. maximum PI for each of the five iterations: [$0.4374, 0.4467, 0.4180, 0.3904, 0.3746$];

2. corresponding $\alpha$s for each of the 5 iterations: [$19.86, 19.84, 15.64, 13.68, 12.44$];

3. best $\alpha$: $12.44$;

4. RMSE on validation set and test set based on the optimized alpha: $0.2501, 0.2826$.

Yes, from the results we can see that, the RMSE for validation set 0.2501 < 0.2700 (from Q4), the model is improved.

The observation noise may come from the optimizer's behavior. The RMSE we got is not necessarily the real optima we expect in this circumstance, because there may be different solver for minimizer. And the solver we use, "L-BFGS-B", may be experiencing early stops, because the message we retrieve from the minimize result is "STOP: TOTAL NO. of ITERATIONS REACHED LIMIT". Therefore, we might get underfitted models or local optimum, instead of the real optimum to get the best RMSE, thus noises may be introduced. Apart from that, different initialization for parameters can also be the noise for mapping $\alpha$ and validation set RMSE.

6. **What next? [20 marks]** Try to improve regression performance beyond the methods we have tried so far. Explain what you tried, why you thought it might work better, how you evaluated your idea, and what you found.
   
   *Do not write more than 300 words for this part*, not including your code (which you do need to include). The bulk of the marks available for this part are for motivating and trying something sensible, which can be simple, and evaluating it sensibly.
   
   The constraints on allowable code given at the start of the assignment still apply. This question is about motivating simple ideas that you can implement from scratch.
   
   Your method does not have to work better to get some marks! However, you need to actually run something and report its results to get some marks. Also, your motivation should be sensible: don’t pick an idea that your existing results already indicate are unlikely to work.
   
   We expect that most students will get less than 10 marks on this question: some of the tasks in this assignment are straightforward and the overall mark on the [common marking scheme](http://web.inf.ed.ac.uk/infweb/student-services/ito/students/common-marking-scheme) should reserve marks in the 80s and 90s for truly outstanding work. However, we advise you not to spend a huge amount of time on this final part. If you are taking 120 credits of courses, this part is worth $0.8\%$ of your mark average. Lots of extra effort might nudge your mark average by $0.2\%$ or less.

   **Answer:**

**Experiment 1**

In [ ]:
np.random.seed(10)
D = X_train.shape[1]
alpha = 12.44

for K in [22, 25, 30, 40, 50]:
    # Create a sensible random initialization of the parameters (uniform(-1, 1) here)
    uni_init = (np.random.uniform(-1, 1, K), np.random.uniform(-1, 1),
                np.random.uniform(-1, 1, (K, D)), np.random.uniform(-1, 1, K))
    # nn fit with sensible random initialization
    params_nn_a = fit_nn(X_train, y_train, alpha, init=uni_init)
    # Report the RMSE of training set
    print(f"Train RMSE for K = {K}, Train RMSE = {get_RMSE_nn(params_nn_a, X_train, y_train)}")
    # Report the RMSE of validation set
    RMSE_nn = get_RMSE_nn(params_nn_a, X_val, y_val)
    print(
        f"Validation RMSE for K = {K}, Val RMSE = {RMSE_nn}")

From the previous experiments, $K$ is another unoptimized parameter. Intuitively, the data is potientially complex, there are more features($373$ features) than $K=20$. The model might perform better with the increasing of $K$ in a range. We manully set $K$ based on the optimized alpha from Q5 as shown above. The results show varied performance with different $K$ values, rather than always increasing as the $K$'s value increases, indicating a balance between capturing complex patterns and avoiding overfitting.

**Experiment 2**

In [ ]:
from scipy.stats import norm
from tqdm import tqdm
np.random.seed(10)

def train_nn_reg_k(X, yy, K):
    # generate init based on K
    init = (np.random.uniform(-1, 1, K), np.random.uniform(-1, 1),
            np.random.uniform(-1, 1, (K, D)), np.random.uniform(-1, 1, K))
    # fit the neural network based on new init
    params = fit_nn(X_train, y_train, 12.44, init)
    # calculate the RMSE on X and yy
    return get_RMSE_nn(params, X, yy)
def bayes_optim_k(k_rest, k_obs, RMSE_base, init=pretrain_init, num_iter=5):
    # Obtain the y values for the beginning observed alphas
    y_obs = []
    for i in tqdm(range(len(k_obs))):
        y_obs.append(np.log(RMSE_base) -
                     np.log(train_nn_reg_k(X_val, y_val, k_obs[i])))

    # Create PIs to record the max PI for each iteration
    PIs = []
    for i in tqdm(range(num_iter)):
        # Calculate the gaussian process prosterior for the rest K
        rest_mu, rest_cov = gp_post_par(k_rest, k_obs, np.array(y_obs))

        # Obtain the PI values for the unobserved alphas
        PI = getPI(rest_mu, rest_cov, y_obs)

        # pick the next K accordding to the PI values for each of the unobserved Ks
        # and update k_obs, k_rest, and PIs
        k_next_id = np.argmax(PI)
        k_obs = np.append(k_obs, k_rest[k_next_id])
        k_rest = np.delete(k_rest, k_next_id)
        PIs.append(PI[k_next_id])

        # Observe the y value for the picked alpha
        y_obs.append(np.log(RMSE_base) -
                     np.log(train_nn_reg_k(X_val, y_val, k_obs[-1])))

    # Retrieve the best alpha according by finding which gives the maximum y_obs
    # assert len(k_obs) == len(y_obs)
    best_k = k_obs[np.argmax(y_obs)]

    # a_obs[3:] corresponds to the newly added alphas
    return best_k, PIs, k_obs[3:]
# Create the possible Ks on the specific range
Ks = np.arange(1, 100, 1) 
# randomly select three beginning Ks from Ks
K_obs = np.random.choice(Ks, 3, replace=False)
# Report the beginning Ks
print("Beginning Ks: ", K_obs)
# Delete the observed Ks from the unobserved ones to get K_rest
K_rest = np.setdiff1d(Ks, K_obs)
# Do bayesian optimization on the given setting (RMSE_nn_val, uni_init from Q4)
RMSE_base = 0.2750728638143858
best_K, PIs, new_Ks = bayes_optim_k(
    K_rest, K_obs, RMSE_base, init=pretrain_init, num_iter=5)
# Report the maximum probability of improvement together with its Ks
# for each of the five iterations
print("PIs:         ", PIs)
print("New Ks:  ", new_Ks)
# Report the best alpha
print("Best K:  ", best_K)

# Report the RMSE on validation set and test set based on the optimized K
print(train_nn_reg_k(X_val, y_val, best_K))
print(train_nn_reg_k(X_test, y_test, best_K))

We then applied Bayesian optimization to $K$, initializing with three random $K$s. Using the maximum PI method, we obtained an optimized $K$ 23 and get the final validation RMSE was $0.2539$. This didn’t improve upon the single alpha optimization method, which might due to the absence of pretrained initialization in $K$’s optimization and the limited number of iterations. Despite this, optimizing hyperparameters during training remains a potential method for performance improvement.

**Experiment 3**

In [ ]:
def rbf_features(X, centers, h):
    # RBFs (x-c)^2
    c = np.sum((X[:, np.newaxis] - centers)**2, axis=-1)
    # Apply the RBF transformation
    return np.exp(-c/(h**2))

# Choose 30 centers for the RBFs
centers = X_train[np.random.choice(X_train.shape[0], 30, replace=False)]
# Choose a K value
h = 10

# Apply the RBF transformation
X_transformed = rbf_features(X_train, centers, h)
X_transformed_val = rbf_features(X_val, centers, h)

To further improve the accuracy, we also tried to apply feature engineering techinuqe method RBFs. We set $30$ radial functions and randomly picked $30$ data points as the center and set $h=10$. After training with the RBFs on the nn model, the RMSE on validation data is $0.4330$, higher than the simple nn model. This could be due to the relatively small number of RBFs and non-optimal $h$. In our experiments, large RBFs can also increase data scale and computational cost. Future exploration could focus on optimizing RBF hyperparameters or applying new feature engineering methods.

[1]: Some of the description on the UCI webpage doesn’t seem to match the dataset. For example there are 97 unique patient identifiers in the dataset, although apparently there were only 74 different patients.↩